# ANASTASIA

This notebook describes the city of ANASTASIA, which writes SiPM responses to montecarlo events produced by geant.

In [4]:
from __future__ import print_function

from glob       import glob
from time       import time

import sys
import numpy  as np
import tables as tb
import pandas as pd


sys.path.append('../../../IC-1')
import invisible_cities.reco.tbl_functions as tbl
from invisible_cities.cities.base_cities               import DetectorResponseCity
from invisible_cities.core.configure                   import configure, \
     print_configuration, \
      read_config_file
from invisible_cities.core.detector_response_functions import HPXeEL,  \
     gather_montecarlo_hits, \
     generate_ionization_electrons, \
     diffuse_electrons, \
     bin_EL, \
     SiPM_response
from invisible_cities.core.detector_geometry_functions import TrackingPlaneBox, \
     MiniTrackingPlaneBox
from invisible_cities.core.system_of_units_c           import units
from invisible_cities.cities.anastasia                 import Anastasia

input_files = ['../../../se_1M_v0_08_07/hdf5_NEXT_NEW_se_1M_v0_08_07_0.h5']
output_file = '../../../S2s/SE_no_diff_no_smear_no_noise3.h'

### Brief overview:
1)  Anastasia receives input files containing montecarlo true tracks of events in the detector

-)  For each input file:

    2) Anastasia gathers all the hits for all the events in the file
    
    -) For every event:
    
         -) For every hit:
         
             3) Anastasia generates all the ionization electrons produced by a hit

             4) Diffuses the ionization electrons to the EL

             5) Finds a 3d box (in x,y,z=time) of SiPMs expected to detect photons produced in EL by these e-
             
             -) For each ionization electron:
             
                 6) Computes fraction of gain each z-slice of SiPMs expected to receieve from e-
                 
                 7) Computes distance boudaries between which photons are cast to SiPMs as e- is crossing EL 

                 8) Computes responses of these SiPMs to each electron (summing responses across electrons/hits)

             9) Saves the SiPM responses to the entire event in a pytables Earray. 

"When you are in the heart of Anastasia one morning your desires waken all at once and surround you. The city appears to you as a whole where no desire is lost and of which you are a part, and since it enjoys everything you do not enjoy, you can do nothing but inhabit this desire and be content. Such is the power, sometimes called malignant, sometimes benign, that Anastasia, the treacherous city, possesses; if for eight hours a day you work as a cutter of agate, onyx, chrysoprase, your labor which gives form to desire takes from desire its form, and you believe you are enjoying Anastasia wholly when you are only its slave."

## A walkthrough Anastasia

First, lets define Anastasia's input and output.

#### Input:
A list of input files.  
The input files are hdf5 files.  
Each hdf5 file contains a pytable containing all the 'hits' for all the events.  
For our purposes, a hit is set of coordinates (x,y,z) within the active region and an energy value


One of these pytables looks something like this:

In [48]:
f_input = tb.open_file(input_files[0], 'r')
table   = f_input.root.MC.MCTracks
# Display a chunk of the pytable
srow  = 280
drows = 5
for i, pos, energy in zip(table[srow: srow + drows]['event_indx'], 
                          table[srow: srow + drows]['hit_position'], 
                          table[srow: srow + drows]['hit_energy']): print([i, list(pos), energy])
    
# ev,     [hit_position (mm)]       , hit_energy (MeV)
f_input.close()

[2, [-5.8504729, 1.6107459, 127.66513], 0.0072159208]
[2, [-5.8618598, 1.594366, 127.6683], 0.013383994]
[2, [-5.8641238, 1.5947694, 127.66937], 0.0045467331]
[3, [-10.80418, 27.498699, 422.33588], 0.0077296947]
[3, [-11.235997, 26.606546, 422.25906], 0.0073481626]


Notice, pytable is organized such that each row is a distinct hit, and the various columns describe the hit. Column event_indx describes which event the hit belongs to. Column hit_position describes the hit's position inside the active region. Column hit_energy describes the amount of energy deposited in the hit by the energetic electron. 

Ultimately, it is convenient that each row in the pytable is a distinct hit, since ultimately Anastasia wants to extract all the hits from all the events in the input files, however, as you can see above, the events are not separated for us by pytables. So, as we are collecting hits in an event, we must notice if the value in event_indx changes to know that we have begun collecting hits in a new event. 

I should also mention the pytable actually contains more columns than those I have shown, but event_indx, hit_position, and hit_energy are the only ones we will pay attention to for now, because they alone matter to Anastasia.

In [35]:
# To satisfy your curiousity, these are all the columns in the pytable
table.description

{
  "event_indx": Int32Col(shape=(), dflt=0, pos=0),
  "mctrk_indx": Int16Col(shape=(), dflt=0, pos=1),
  "particle_name": StringCol(itemsize=10, shape=(), dflt=b'', pos=2),
  "pdg_code": Int16Col(shape=(), dflt=0, pos=3),
  "initial_vertex": Float32Col(shape=(3,), dflt=0.0, pos=4),
  "final_vertex": Float32Col(shape=(3,), dflt=0.0, pos=5),
  "momentum": Float32Col(shape=(3,), dflt=0.0, pos=6),
  "energy": Float32Col(shape=(), dflt=0.0, pos=7),
  "nof_hits": Int16Col(shape=(), dflt=0, pos=8),
  "hit_indx": Int16Col(shape=(), dflt=0, pos=9),
  "hit_position": Float32Col(shape=(3,), dflt=0.0, pos=10),
  "hit_time": Float32Col(shape=(), dflt=0.0, pos=11),
  "hit_energy": Float32Col(shape=(), dflt=0.0, pos=12)}

###### TODO:
We need to decide how what want to store our output. Do we want to store in EArrays? If this is the plan we are going to need to add some kind of descriptor, ex: (x pitch = a, y_pitch = b, time bins were 2 mus). It would also be nice if this discription were did not have to be read by a human. 

#### Output
The output is very simple -- too simple and we will need to change it. It is an hdf5 file containing a pytables EArray, containing all the SiPM responses to all the events. 

In [44]:
f_output = tb.open_file(output_file, 'r')
print(f_output)
SiPM_resp = f_output.root.SiPM_resp
f_output.close()

../../../S2s/SE_no_diff_no_smear_no_noise3.h (File) ''
Last modif.: 'Mon Mar 13 15:20:11 2017'
Object Tree: 
/ (RootGroup) ''
/SiPM_resp (EArray(5, 48, 48, 266), shuffle, zlib(4)) ''



Above you can hopefully see inside the output file there is an EArray of shape (5, 48, 48, 266).  

In [50]:
# SiPM_resp.shape = (NEVENTS, SiPMs along x dimension, SiPMs along y dimension, time bins)
f_output.close()

There are 5 events  
The SiPM plane is a 48x48 square  
The 4th dimension here is time. The SiPM responses are binned in 266 time bins.

### How does Anastasia work?

In [51]:
A = Anastasia(input_files, output_file,
        hpxe    = HPXeEL(),
        tpbox   = TrackingPlaneBox(),
    
        # Parameters added at this level
        NEVENTS = 5,
        w_dim   = (8, 8, 4))

Notice, Anastasia has two required inputs:  
a list of input files and an output file,   
Anastasia also has a few kwargs which can be configured or set by default:

1) NEVENTS is the number of events over which Anastasia will run.

2) hpxe is an instance of a HPXeEL. hpxe contians information regarding the physical 
conditions of the detector, and (specifically for Anastasia) how these conditions 
will affect the behavior of ionization electrons in the active region and as they 
cross the EL.

3) tpbox is an instance of a TrackingPlaneBox. A tracking plane box is a class 
that inherits from Box. It has attributes describing its shape, size, and absolute
position. It also has attributes describing the pitch of the SiPMs their positions,
and their response to an event. Note tpbox is a box (and not a plane) because SiPMs
are responding in time bins. 

4) w_dim is the dimensions of a 3d box of SiPMs (in x, y, and z=time) expected to
respond to the photons produced by the ionization electrons produced by a hit. We 
will come back to exactly what this means. w_dim describes the size of a 
MiniTrackingPlaneBox that will be found within the larger TrackingPlaneBox to 
compute SiPM responses to photons produced by ionization electrons produced by a 
single hit. We will take a closer look at w_dim in a bit.

Note: 
Anastasia ineherits from DetectorResponseCity, where hpxe and tpbox are saved as
attributes. NEVENTS and w_dim are her only unshared parameters.

Anastasia has one method, run. Using configured or default parameters, classes hpxe and tpbox, along with varias functions in Core/DetectorResponseFunctions, run computes the SiPM responses for NEVENTS in the input files writes
the responses to file. 

In [3]:
A.run()

../../../S2s/SE_no_diff_no_smear_no_noise3.h (File) ''
Last modif.: 'Mon Mar 13 15:20:11 2017'
Object Tree: 
/ (RootGroup) ''
/SiPM_resp (EArray(5, 48, 48, 266), shuffle, zlib(4)) ''

